Домашняя работа по теме "Основы веб-скрапинга"

In [ ]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

import time

import numpy as np

Задание 1. Необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [ ]:
queries = ['python', 'анализ данных']

In [ ]:
def get_posts_from_habr(some_queries):
  """
  Возвращает датафрейм вида <дата> - <заголовок> - <ссылка на материал> по результатам поиска на habr.com
  type(some_queries) = list
  """
  df = pd.DataFrame()

  # перебираем запросы из списка
  for query in some_queries:
    params = {
        'q': query,
        'target_type': 'posts',
        'order': 'relevance'
        }
    #отправляем запрос к сайту и преобразовываем результат в нужный формат
    result = requests.get('https://habr.com/ru/search/', params=params)
    result_html = BeautifulSoup(result.text)
    time.sleep(0.1)
    #достаем с сайта все посты, отвечающие запросу
    posts = result_html.find_all('article', class_='tm-articles-list__item')

    #из каждого поста достаем нужную информацию
    for post in posts:
      date = post.find('time').get('title')[:10]   #отсекаем время публикации поста
      #обработка ошибок при обращении к посту с иной разметкой (последний пост по запросу "анализ данных")
      try:
        header = post.find('a', 'tm-article-snippet__title-link').text
        link = post.find('a', 'tm-article-snippet__title-link').get('href')
      except:
        header = post.find('a', 'tm-megapost-snippet__card').text
        link = post.find('a', 'tm-megapost-snippet__card').get('href')
      full_link = 'https://habr.com' + link
      #создаем строки датафрейма
      row = {'дата': date, 'заголовок': header, 'ссылка на материал': full_link}
      df = pd.concat([df, pd.DataFrame([row])])

  #исключаем дубликаты
  df.drop_duplicates(subset='ссылка на материал', keep='first', inplace=True)

  return df

In [ ]:
posts_from_habr = get_posts_from_habr(queries)

In [ ]:
posts_from_habr.info()
#результат поиска на сайте - 20 постов
#судя по количеству строк, дубликатов в датафрейме не было

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 0
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   дата                40 non-null     object
 1   заголовок           40 non-null     object
 2   ссылка на материал  40 non-null     object
dtypes: object(3)
memory usage: 1.2+ KB


In [ ]:
posts_from_habr.head()

,дата,заголовок,ссылка на материал
0,2022-01-20,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/company/southbridge/news/t...
0,2021-12-13,Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/company/southbridge/news/t...
0,2020-04-21,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/
0,2021-07-06,Python Community Meetup 8/07: видео и материал...,https://habr.com/ru/company/raiffeisenbank/new...
0,2022-01-13,Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/company/southbridge/news/t...


In [ ]:
posts_from_habr.tail()

,дата,заголовок,ссылка на материал
0,2014-11-23,Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/243967/
0,2017-06-27,Использование Python и Excel для обработки и а...,https://habr.com/ru/company/otus/blog/331746/
0,2014-09-14,Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/236757/
0,2015-01-18,Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/248165/
0,2017-12-25,Новый подход к спортивному анализу данных: как...,https://habr.com/ru/article/344806/


Задание 2. Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;

в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:

<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [ ]:
def get_posts_from_habr_upgraded(some_queries, count_pages):
  """
  По результатам поиска на habr.com возвращает датафрейм вида:
  <дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
  type(some_queries) = list
  count_pages = количество страниц поисковой выдачи
  """
  df = pd.DataFrame()
  links = []
  # перебираем запросы из списка
  for query in some_queries:
    params = {
        'q': query,
        'target_type': 'posts',
        'order': 'relevance'
        }
    #перебираем страницы поисковой выдачи и собираем все ссылки на посты
    for page in range(1, count_pages + 1):
      url = 'https://habr.com/ru/search/page' + str(page) + '/'
      result = requests.get(url, params=params)
      result_html = BeautifulSoup(result.text)
      time.sleep(0.1)
      posts = result_html.find_all('article', class_='tm-articles-list__item')
      for post in posts:
        try:
          link = post.find('a', 'tm-article-snippet__title-link').get('href')
        except:
          link = post.find('a', 'tm-megapost-snippet__card').get('href')
        full_link = 'https://habr.com' + link
        if full_link not in links:
          links.append(full_link)
  #переходим по каждой ссылке и собираем нужную информацию
  for link_ in links:
    res_html = BeautifulSoup(requests.get(link_).text)
    time.sleep(0.1)
    date = res_html.find('time').get('title')[:10]   #отсекаем время публикации поста
    try:
      header = res_html.find('h1', 'tm-article-snippet__title').text
      full_text = res_html.find('div', id='post-content-body').text
    except:
      header = np.NAN
      full_text = np.NAN
    likes = res_html.find('span', 'tm-votes-meter__value').text.strip('+')
    #создаем строки датафрейма
    row = {'дата': date, 'заголовок': header, 'ссылка на материал': link_, 'текст материала': full_text, 'количество лайков': likes}
    df = pd.concat([df, pd.DataFrame([row])])

  return df

In [ ]:
posts_from_habr_upgraded = get_posts_from_habr_upgraded(queries, 4)

In [ ]:
#одна статья дублировалась
posts_from_habr_upgraded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 0
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   дата                159 non-null    object
 1   заголовок           158 non-null    object
 2   ссылка на материал  159 non-null    object
 3   текст материала     158 non-null    object
 4   количество лайков   159 non-null    object
dtypes: object(5)
memory usage: 7.5+ KB


In [ ]:
# на сайте какая-то ошибка с доступом с 9 статье
posts_from_habr_upgraded.head(10)

,дата,заголовок,ссылка на материал,текст материала,количество лайков
0,2022-01-20,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/company/southbridge/news/t...,"Курс нацелен дать максимальную пользу, поэтому...",10
0,2021-12-13,Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/company/southbridge/news/t...,21 декабря Слёрм проведёт открытый урок «ChatO...,9
0,2020-04-21,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/,"\r\n20 апреля 2020 года, спустя почти десять л...",19
0,2021-07-06,Python Community Meetup 8/07: видео и материал...,https://habr.com/ru/company/raiffeisenbank/new...,"Первый открытый онлайн-митап сообщества, для к...",3
0,2022-01-13,Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/company/southbridge/news/t...,19 января Слёрм проведёт открытый урок «Пишем ...,10
0,2020-12-04,Python как компилируемый статически типизирова...,https://habr.com/ru/news/t/531402/,По данным широко известного в узких кругах Tio...,16
0,2022-03-08,Вышел мартовский релиз расширения Python для V...,https://habr.com/ru/news/t/654707/,Вышел выпуск расширения Python для Visual Stud...,0
0,2020-03-03,В начале этого года Python сместил Java и стал...,https://habr.com/ru/company/itsumma/news/t/490...,"Согласно отчету RedMonk за январь 2020 года, P...",31
0,2022-02-18,NaN,https://habr.com/ru/company/epam_systems/news/...,NaN,2
0,2022-08-19,Осталась неделя до старта 4 потока Python для ...,https://habr.com/ru/company/southbridge/news/t...,29 августа в последний раз в этом году запусти...,5


Домашняя работа выполнена.